# LSTM Model Training for Data Generation

This notebook presents the training of a LSTM model to generate electric consuption data.

### Data Preprocessing

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# fix random seed for reproducibility
np.random.seed(26072001)

In [ ]:
# import the data set 
industrial = pd.read_csv('data/industrial_total.csv')

The dataset has 3 columns. It is an aggregation of the whole building.
* `cumulative` These values represent the cumulative consumption of the building. The unit is kW/h
* `difference` These values represent the difference between each cumulative value. 
* `time` The according timestamp in format yyyy-mm-dd'T'hh:mm:ss+Z


In [ ]:
industrial.head()

We will use the `difference` column as our base data. We will aggregate this data per hour, creating a mean that reprensents the minutely consumption per hour. The unit then becomes kW/h/m. 

In [ ]:
# aggregate the data to have a mean
# 'mean' -> average over the chosen period
# 'first' -> select the first timestamp 
d = {'difference': 'mean', 'time': 'first'}

# create the hourly dataset that creates a mean over every 60 values
hourly = industrial.groupby(industrial.index // 60).agg(d)
# add a column to associate an arbitrary hour number for each value (kind of an index)
hourly['hour'] = hourly.index + 1
hourly = hourly.rename(columns={"difference": "mean"})

In [ ]:
hourly.head()

In [ ]:
# plot of the first 500 values
hourly.head(500).plot(x = 'hour', 
                        y = 'mean',
                        title = 'Average Minutely Consumption of \n an Industrial Building per Hour',
                        xlabel = 'Hour',
                        ylabel = 'kW/h')

plt.tight_layout()

### LSTM Model

In [ ]:
# create a pandas dataframe and select the mean of the hourly dataset as a first base column
dataset = pd.DataFrame()
dataset['mean'] = hourly['mean']

In [ ]:
# normalize (scale) the dataset to the range (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split the data to create the train and test sets
train_size = int(len(dataset) * 0.7) # 70% sample for training
test_size = len(dataset) - train_size # testing
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test)) # check the sizes of each set

The next step is to transform our base data (hourly scaled mean) into a "supervised" dataset with lookback. The lookback is the number of previous values taken into account for the prediction. Each `nth` column will be the `nth-1` column with a delay of 1.

In [ ]:
# function to transform an array of values into a supervised dataset with lookback parameter
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# create train and test sets
look_back = 12
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

As we can see, each value appears in each column with a lag of one.

In [ ]:
pd.DataFrame(trainX)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# add a validation sample of 33%
# input the training in 'history' variable to look back at the loss
history = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2, validation_split=0.33)

In [ ]:
model.summary()

Now, we take a look at the evolution of the loss. The metric is the mean squared error.

In [ ]:
# check the saved metric from the model training
print(history.history.keys())

In [ ]:
# evolution plot of the mse
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test (validation)'], loc='upper left')
plt.show()

### Predictions

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# unscale predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# compute RMSE
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
# zoom on the first 500 predictions on the training data
plt.plot(pd.DataFrame(scaler.inverse_transform(dataset)).head(500))
plt.plot(pd.DataFrame(trainPredictPlot).head(500))
plt.legend(['True value', 'Train prediction'])

In [ ]:
# zoom on the first 500 predictions on the testing data
plt.plot(pd.DataFrame(scaler.inverse_transform(dataset)).tail(500))
plt.plot(pd.DataFrame(testPredictPlot).tail(500), color = 'lightgreen')
plt.legend(['True value', 'Test prediction'])

In [ ]:
# if the model wants to be saved
model.save("models/lstm_lookback_12")